## image classification using tensorflow for handwashing    
### methodology:
- seperate into train and test data
- seperate into images
- Normalization, Reshape and Label Encoding
- optimizer, loss function
- evaluate

In [27]:
import os
import random
import cv2
import tensorflow as tf
import shutil

In [88]:
input_dir = '/Users/andrewbahsoun/Documents/computer_science/sci-250/hand-washing/data/HandWashDataset/'
output_frames_dir_test = '/Users/andrewbahsoun/Documents/computer_science/sci-250/hand-washing/data/outputFrames/test'
output_frames_dir_train = '/Users/andrewbahsoun/Documents/computer_science/sci-250/hand-washing/data/outputFrames/train'

steps = ['nostep0', 'Step_1', 'Step_2', 'Step_3', 'Step_4', 'Step_5', 'Step_6', 'Step_7', 'Step_8', 'Step_9', 'Step_10', 'Step_11', 'Step_12' ]


In [89]:
all_file_names_dict = {
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
    11: [],
    12: []
}


for step in range(1, 13):
    
    for name in os.listdir(os.path.join(input_dir, steps[step])):
        # Open file
        with open(os.path.join(input_dir, steps[step], name)) as f:
            all_file_names_dict[step].append(name)



### get all videos into frames

In [86]:
def get_frames_from_video(directory, filename, step, output_frames_dir):
    # Creating a VideoCapture object to read the video
    cap = cv2.VideoCapture(os.path.join(directory, steps[step], filename))

    is_success, image = cap.read()
    frame_number = 0

    while is_success:
        out_filename = "frame_{}_{}.jpg".format(frame_number, os.path.splitext(filename)[0])
        save_path_and_name = os.path.join(output_frames_dir, out_filename)
        cv2.imwrite(save_path_and_name, image)
        is_success, image = cap.read()
        frame_number += 1


In [95]:
counter = 0
test_ratio = 0.3

for step in range(1,13):
    counter = 0
    for video in all_file_names_dict[step]:
        if (video != ".DS_Store"):

            if ((len(all_file_names_dict) * (1-test_ratio) ) < counter):
                #train data
                get_frames_from_video(input_dir, video, step, output_frames_dir_train)
            else:
                #test data
                get_frames_from_video(input_dir, video, step, output_frames_dir_test)
            counter += 1
    

#### debugging purposes 
this will print out all the files in the directory onto a file

In [31]:
def find_num_of_videos_per_step(directory, output_filename):
    with open(output_filename, 'a') as f:  # Open the file in append mode
        for name in os.listdir(directory):
            f.write(name + '\n')  # Write each name on a new line


In [ ]:
find_num_of_videos_per_step(output_frames_dir_test, 'test_videos')
find_num_of_videos_per_step(output_frames_dir_train, 'train_videos')

### get photos into directories by class

In [52]:
def move_video_into_subdirectory_onedigit(directory, output_dir, step):
    for name in os.listdir(directory):
            if ("A_0" + str(step)) in name:
                shutil.move(os.path.join(directory, name), output_dir)
    

In [53]:
def move_video_into_subdirectory_twodigit(directory, output_dir, step):
    for name in os.listdir(directory):
            if ("A_" + str(step)) in name:
                shutil.move(os.path.join(directory, name), output_dir)

In [57]:
#moving all test photos step(1-9) into their respective directories
for step in range(1, 10):
    move_video_into_subdirectory_onedigit(output_frames_dir_test, os.path.join(output_frames_dir_test,('step_' + str(step))), step)

In [58]:
#moving all test photos step(10-12) into their respective directories
for step in range(10, 13):
    move_video_into_subdirectory_twodigit(output_frames_dir_test, os.path.join(output_frames_dir_test,('step_' + str(step))), step)

In [67]:
#moving all train photos step(1-9) into their respective directories
for step in range(1, 10):
    move_video_into_subdirectory_onedigit(output_frames_dir_train, os.path.join(output_frames_dir_train,('step_' + str(step))), step)

In [73]:
#moving all train photos step(10-12) into their respective directories
for step in range(10, 13):
    move_video_into_subdirectory_twodigit(output_frames_dir_train, os.path.join(output_frames_dir_train,('step_' + str(step))), step)

### load dataset

In [81]:
# Load the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    output_frames_dir_train,
    image_size=(224, 224),  # Resize images 
    batch_size=32,          # Number of images to return in each batch
    label_mode='categorical', # Multi-class classification
    color_mode='rgb' # using 3 channels
)

Found 155157 files belonging to 12 classes.


In [80]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    output_frames_dir_test,
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    color_mode='rgb'
)

Found 79695 files belonging to 12 classes.


In [86]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# Initialize the Sequential model
model = Sequential()

# Add layers to the model
model.add(Conv2D(filters=8, kernel_size=(5,5), padding='Same',
                 activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))  # Assuming 12 classes for classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 8)    │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 56, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 28, 28, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         3,084 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,218,700 (12.28 MB)

 Trainable params: 3,218,700 (12.28 MB)

 Non-trainable params: 0 (0.00 B)

In [87]:
# Define the optimizer
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [88]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [89]:
epochs = 5  # 
batch_size = 500

In [92]:
model.fit(train_dataset, validation_data=test_dataset, epochs = 5)

Epoch 1/5
4849/4849 ━━━━━━━━━━━━━━━━━━━━ 1458s 300ms/step - accuracy: 0.6679 - loss: 2.5602 - val_accuracy: 0.2451 - val_loss: 7.1910
Epoch 2/5
1528/4849 ━━━━━━━━━━━━━━━━━━━━ 10:32 190ms/step - accuracy: 0.8985 - loss: 0.2815

KeyboardInterrupt: 

In [93]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

2491/2491 ━━━━━━━━━━━━━━━━━━━━ 112s 45ms/step - accuracy: 0.2502 - loss: 7.2107
Test Loss: 7.234890937805176
Test Accuracy: 0.25113242864608765


In [ ]:
# Get predictions from the model
predictions = model.predict(test_dataset)

# Convert predictions to class labels
predicted_classes = tf.argmax(predictions, axis=1).numpy()

# Get the true labels from the dataset
true_classes = []
for images, labels in test_dataset:
    true_classes.extend(tf.argmax(labels, axis=1).numpy())

# Compare the predictions to the actual labels
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(true_classes, predicted_classes))

print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()